In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,62,2024-03-06,3,111.491803,96.754098,0.541852,12.640984,26.004918,0.226115,114.565574,...,11.793443,19.939344,0.172000,111.332787,0,3,2,3,22,2024
1,61,2024-03-06,-1,117.166667,97.051667,0.565400,11.188333,24.381667,0.215517,120.491667,...,11.175000,24.688333,0.205567,113.430000,0,3,2,2,13,2024
2,63,2024-03-06,1,106.064516,97.654839,0.514823,12.782258,22.219355,0.184177,108.127419,...,9.950000,24.925000,0.238467,119.496667,0,3,2,2,15,2024
3,61,2024-03-06,-1,114.350000,97.068333,0.558733,11.598333,24.131667,0.185083,117.080000,...,10.836667,27.643333,0.216800,119.025000,0,3,2,3,6,2024
4,62,2024-03-06,1,111.672131,95.398361,0.530541,10.559016,24.559016,0.188705,115.160656,...,12.974194,28.711290,0.212548,117.398387,0,3,2,2,4,2024
5,63,2024-03-06,6,121.354839,100.225806,0.573645,10.929032,21.816129,0.222581,120.469355,...,12.080000,27.445000,0.185417,118.431667,0,3,2,2,17,2024
6,62,2024-03-06,-1,120.901639,99.121311,0.576098,10.386885,20.657377,0.203197,121.163934,...,12.513333,26.841667,0.193067,110.111667,0,3,2,2,21,2024
7,61,2024-03-06,-1,118.050000,98.903333,0.563000,11.263333,23.915000,0.172383,118.080000,...,11.815873,19.676190,0.210540,116.036508,0,3,2,2,26,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-06,ORL,WAS,1.350877,3.300000,1
1,2024-03-06,LAC,HOU,1.387597,3.100000,1
2,2024-03-06,MEM,PHI,2.800000,1.458716,0
3,2024-03-06,CLE,ATL,2.200000,1.704225,0
4,2024-03-06,CHI,UTA,1.588235,2.420000,0
5,2024-03-06,MIL,GSW,2.360000,1.617284,1
6,2024-03-06,OKC,POR,1.100000,7.500000,1
7,2024-03-06,SAC,LAL,2.140000,1.740741,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-06,ORL,WAS,1.350877,3.300000,1
1,2024-03-06,LAC,HOU,1.387597,3.100000,1
2,2024-03-06,MEM,PHI,2.800000,1.458716,0
3,2024-03-06,CLE,ATL,2.200000,1.704225,0
4,2024-03-06,CHI,UTA,1.588235,2.420000,0
5,2024-03-06,MIL,GSW,2.360000,1.617284,0
6,2024-03-06,OKC,POR,1.100000,7.500000,1
7,2024-03-06,SAC,LAL,2.140000,1.740741,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-06,ORL,WAS,1.350877,3.300000,1
1,2024-03-06,LAC,HOU,1.387597,3.100000,1
2,2024-03-06,MEM,PHI,2.800000,1.458716,0
3,2024-03-06,CLE,ATL,2.200000,1.704225,0
4,2024-03-06,CHI,UTA,1.588235,2.420000,0
5,2024-03-06,MIL,GSW,2.360000,1.617284,0
6,2024-03-06,OKC,POR,1.100000,7.500000,1
7,2024-03-06,SAC,LAL,2.140000,1.740741,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-06,ORL,WAS,1.350877,3.300000,1
1,2024-03-06,LAC,HOU,1.387597,3.100000,1
2,2024-03-06,MEM,PHI,2.800000,1.458716,0
3,2024-03-06,CLE,ATL,2.200000,1.704225,0
4,2024-03-06,CHI,UTA,1.588235,2.420000,0
5,2024-03-06,MIL,GSW,2.360000,1.617284,0
6,2024-03-06,OKC,POR,1.100000,7.500000,1
7,2024-03-06,SAC,LAL,2.140000,1.740741,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-06,ORL,WAS,1.350877,3.300000,1.0
1,2024-03-06,LAC,HOU,1.387597,3.100000,1.0
2,2024-03-06,MEM,PHI,2.800000,1.458716,0.0
3,2024-03-06,CLE,ATL,2.200000,1.704225,0.0
4,2024-03-06,CHI,UTA,1.588235,2.420000,0.0
5,2024-03-06,MIL,GSW,2.360000,1.617284,1.0
6,2024-03-06,OKC,POR,1.100000,7.500000,1.0
7,2024-03-06,SAC,LAL,2.140000,1.740741,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-06,ORL,WAS,1.350877,3.300000,1.0
1,2024-03-06,LAC,HOU,1.387597,3.100000,1.0
2,2024-03-06,MEM,PHI,2.800000,1.458716,0.0
3,2024-03-06,CLE,ATL,2.200000,1.704225,0.0
4,2024-03-06,CHI,UTA,1.588235,2.420000,0.0
5,2024-03-06,MIL,GSW,2.360000,1.617284,1.0
6,2024-03-06,OKC,POR,1.100000,7.500000,1.0
7,2024-03-06,SAC,LAL,2.140000,1.740741,0.0
